# Diurnal Rhythm of the Human Plasma Proteome

This notebook and datasets supports the manuscript “Diurnal Rhythm of the Human Plasma Proteome,” which investigates how protein concentrations in human plasma fluctuate over a 24-hour period. Using high-throughput mass spectrometry, we analyzed 208 high-quality plasma samples collected every three hours from 24 healthy individuals under tightly controlled conditions. Out of 523 quantified proteins, 138 (~26%) showed significant diurnal rhythmicity. These proteins were enriched in specific tissues, particularly the liver and platelets, and were involved in key biological pathways including hemostasis, immune signaling, and metabolism. Importantly, 36 clinically relevant biomarkers displayed time-of-day-dependent variation, highlighting the need to incorporate temporal factors in diagnostic and research protocols. This dataset offers valuable insights into circadian regulation of the plasma proteome and provides a resource for researchers studying time-sensitive biomarkers and physiological processes.

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
from pimmslearn.plotting.defaults import color_model_mapping
import pimmslearn.plotting.data
import pimmslearn.sampling
from pimmslearn.sklearn.ae_transformer import AETransformer
#from inmoose.pycombat import pycombat_norm
from scipy.stats import variation
from CosinorPy import cosinor
from CosinorPy.cosinor import fit_me
import scipy.stats as stats
from scipy.stats import pearsonr
from scipy.stats import fisher_exact


from statsmodels.stats.multitest import multipletests


%matplotlib inline
init_notebook_mode()

In [ ]:
def clean_column_names(df, return_format='name'):
    """
    Clean column names of a DataFrame to extract sample names and filenames.
    Parameters:
    df (pd.DataFrame): DataFrame with columns to be cleaned.
    return_format (str): Format of the returned DataFrame, either 'name' or 'filename'.
    Returns:
    pd.DataFrame: DataFrame with cleaned column names.
    """
    cols = pd.DataFrame(df.columns.tolist(), columns = ['filename'])
    cols['name']= cols.filename.str.split('\\').str[-1].str.split('.raw').str[0]
    cols['sample']=cols.name.str.split('_').str[-1].str.split('.raw').str[0]
    cols.loc[cols.name.isnull(),'name']=cols.loc[cols.name.isnull()]['filename']
    cols.loc[cols['sample'].isnull(),'sample']=cols.loc[cols['sample'].isnull()]['filename']
    cols.loc[0:4,return_format]=cols.loc[0:4,'filename']
    return cols


def filter_missingness(df, feat_prevalence=.2, axis=0):
    """
    Filter features (rows or columns) based on their prevalence in the DataFrame.
    Parameters:
    df (pd.DataFrame): DataFrame to filter.
    feat_prevalence (float): Minimum fraction of non-missing values required for a feature to be retained.
    axis (int): Axis along which to filter (0 for rows, 1 for columns).
    Returns:
    pd.DataFrame: Filtered DataFrame with features that meet the prevalence threshold.
    """
    N = df.shape[axis]
    minimum_freq = N * feat_prevalence
    freq = df.notna().sum(axis=axis)
    mask = freq >= minimum_freq
    print(f"Drop {(~mask).sum()} along axis {axis}.")
    freq = freq.loc[mask]
    if axis == 0:
        df = df.loc[:, mask]
    else:
        df = df.loc[mask]
    return df

def calculate_cv(df, individual_col):
    """
    Calculate the coefficient of variation (CV) of proteins per individual using scipy.stats.variation.
    
    Parameters:
    df (pd.DataFrame): DataFrame where rows are samples, columns are proteins, and one column represents individuals.
    individual_col (str): Name of the column indicating individuals.
    
    Returns:
    pd.DataFrame: A DataFrame with individuals as index and CV values per protein as columns.
    """
    # Exclude the individual column from calculations
    protein_cols = df.columns.difference([individual_col])
    
    # Group by individual and calculate variation for each protein
    cv_values = df.groupby(individual_col)[protein_cols].apply(lambda x: variation(x, axis=0, nan_policy='omit'))
    
    return cv_values


def zscore_by_individual(group, protein_cols):
    """
    Z-score each protein across time for a given individual.
    Parameters:
        group (pd.DataFrame): DataFrame for a single individual with timepoints and protein expression data.
        protein_cols (list): List of protein column names.
    Returns:
        pd.DataFrame: Group with z-scored protein values.
    """
    # Z-score each protein across time for this individual
    for protein in protein_cols:
        group[protein] = (group[protein] - group[protein].mean()) / group[protein].std()
    return group

def analyze_circadian_rhythms(df, time_col, protein_cols, period=24, alpha=0.05):
    """
    Perform cosinor analysis on multiple proteins and apply FDR correction.

    Parameters:
        df (pd.DataFrame): DataFrame with timepoints and protein expression data.
        time_col (str): Name of the column with timepoints.
        protein_cols (list): List of protein column names.
        period (float): Expected circadian period (default: 24 hours).
        alpha (float): Significance level for p-value correction (default: 0.05).

    Returns:
        pd.DataFrame: Results with amplitude, phase, p-value, and adjusted p-value.
    """
    results = []
    
    for protein in protein_cols:
        time = df[time_col]
        expression = df[protein]

        # Fit cosinor model
        cosinorfx = cosinor(time, expression, period=period)
        cosinorfx.fit()

        # Collect results
        results.append({
            "protein": protein,
            "amplitude": cosinorfx.amplitude,
            "phase": cosinorfx.acrophase,
            "p_value": cosinorfx.p_value
        })
    
    # Convert to DataFrame
    results_df = pd.DataFrame(results)

    # Apply Benjamini-Hochberg correction for multiple comparisons
    results_df["adjusted_p_value"] = multipletests(results_df["p_value"], method="fdr_bh")[1]

    # Filter significant circadian proteins
    significant_proteins = results_df[results_df["adjusted_p_value"] < alpha]

    return significant_proteins

def acrophase_hr(phi,A):
    """
    Calculate the acrophase in hours from the phase (phi) and amplitude (A) of a circadian rhythm.
    Parameters:
    phi (float): Phase of the rhythm in radians.
    A (float): Amplitude of the rhythm.
    Returns:
    float: Acrophase in hours.
    """
    omega = 2 * np.pi / 1440  # 24-hour rhythm in minutes
    # Check if the amplitude is negative — flip phase if so
    phi_corrected = phi if A >= 0 else phi + np.pi

    # Calculate the acrophase time
    acrophase_minutes = (-phi_corrected / omega) % 1440
    hours = int(acrophase_minutes // 60)
    minutes = int(acrophase_minutes % 60)
    acrophase_hr = acrophase_minutes / 60
    return acrophase_hr

def mean_ci(series):
    """
    Calculate the mean, 95% confidence interval, median, and interquartile range (IQR) of a series.
    Parameters:
    series (pd.Series or np.ndarray): Input data series.
    Returns:
    pd.Series: A Series containing the mean, lower and upper bounds of the 95% confidence interval, median, and 25th and 75th percentiles (IQR).
    """
    mean = np.mean(series)
    n = len(series)
    std_err = stats.sem(series)  # Standard error = SD / sqrt(n)
    ci = stats.t.ppf(0.975, n-1) * std_err  # 95% CI (two-tailed)
    qt25 = np.percentile(series, 25)
    qt75 = np.percentile(series, 75)
    return pd.Series({'mean': mean, 'CI_lower': mean - ci, 'CI_upper': mean + ci,'median':np.median(series),'q25':qt25,'q75':qt75})

### Read in the data

In [ ]:
# protein data from DIA-NN:
proteome = pd.read_csv('report.pg_matrixCircadian.tsv', sep ='\t')
# sample data from the proteome
overview = clean_column_names(proteome)


In [ ]:
# clan up the overview table:
overview['ID']=overview['sample'].str.split('-').str[0]
overview['day']=overview['sample'].str.split('-').str[1]
overview['timepoint']=overview['sample'].str.split('-').str[2].str.split('.').str[0]
overview['plate']= overview.filename.str.split('CircadianRythm_').str[1].str.split('_').str[0]
# rename empty or pool samples (Quality control):
for col in ['sample','ID']:
    overview.loc[overview.filename.str.contains('empty', case = False),col]='Empty'
    overview.loc[overview.filename.str.contains('pool', case = False),col]='QCpool'
# there are two duplicated samples, rename them:
overview.loc[(overview.timepoint.isnull())&(overview.ID!='Empty'),'sample']=overview.loc[(overview.timepoint.isnull())&(overview.ID!='Empty')].filename.str.split('Plate2_').str[1]
overview.loc[(overview.timepoint.isnull())&(overview.ID!='Empty'),'ID'] = overview.loc[(overview.timepoint.isnull())&(overview.ID!='Empty')]['sample'].str.split('-').str[0]
overview.loc[(overview.timepoint.isnull())&(overview.ID!='Empty'),'day'] = overview.loc[(overview.timepoint.isnull())&(overview.ID!='Empty')]['sample'].str.split('-').str[1]
overview.loc[(overview.timepoint.isnull())&(overview.ID!='Empty'),'timepoint'] = overview.loc[(overview.timepoint.isnull())&(overview.ID!='Empty')]['sample'].str.split('-').str[2].str.split('_').str[0]
# define the sample type:
overview['sample_type']='sample'
overview.loc[overview.filename.str.contains('empty', case = False),'sample_type']='Empty'
overview.loc[overview.filename.str.contains('pool', case = False),'sample_type']='QCpool'


In [ ]:
# prepare the proteome data for analysis:
index_cols = ['filename','sample','ID','day','timepoint','plate','sample_type']
pg_long = proteome.melt(id_vars=proteome.columns[0:4].tolist())
pg_long.rename(columns = {'variable':'filename','value':'LFQ'}, inplace = True)
pg_long.dropna(subset = ['LFQ'], inplace = True)
pg_long =  pd.merge(overview[index_cols], pg_long, on = 'filename')

### Data Processing

In [ ]:
# plot the number of proteins per sample:
count_df = pg_long.fillna('').groupby(index_cols).size().reset_index()
count_df.columns = index_cols+['proteins']
count_df= count_df.sort_values(['sample_type','proteins'], ascending = [False, False])


In [ ]:
# Figure 1C: Number of proteins
fig1a = px.histogram(count_df[count_df.sample_type=='sample'], x="proteins", color="sample_type",
                marginal="box", # or violin, rug
                hover_data=count_df.columns,
                template = 'simple_white')
fig1a.update_layout(bargap = 0.1)
fig1a.update_xaxes(range=[200,count_df.proteins.max()+10])
fig1a.update_yaxes(matches=None)
fig1a.show()  

fig1a.write_image('1a.pdf', width = 500)

#### Filter samples on missingness (below 1.5IQR from 25th quantile)

In [ ]:
sub_df = count_df.loc[count_df.sample_type=='sample']
# Calculate Q1 and Q3 for the 'proteins' column
Q1 = sub_df['proteins'].quantile(0.25)
Q3 = sub_df['proteins'].quantile(0.75)

# Compute the IQR
IQR = Q3 - Q1

# Compute 1.5xIQR
threshold = 1.5 * IQR

# Calculate the bounds for outliers
lower_bound = Q1 - threshold

print(f"Lower Bound: {lower_bound}")

In [ ]:
exclusion_df = count_df.loc[count_df.proteins<=lower_bound]
no_exclusion = exclusion_df[exclusion_df.sample_type!='Empty'].shape[0]
print(f"Number of samples below lower bound: {no_exclusion}")

In [ ]:
# exclude samples before furteher analysis
pg_long = pg_long.loc[pg_long['filename'].isin(exclusion_df['filename'])==False]
pg_long['filename'].unique().shape


#### Filter protein values:

In [ ]:
# format the proteome data for further analysis:
index_cols = pg_long.columns[0:7]
df_wide = pg_long.pivot(index = index_cols, columns='Protein.Group',values = 'LFQ')
df_wide = np.log2(df_wide)
# filter out proteins with too many missing values, only based on sample_type=='sample (not QC or Empty):
pg_wide = df_wide.reset_index()
pg_wide_sample = pg_wide.loc[pg_wide.sample_type=='sample'].drop(index_cols[1:], axis = 1).set_index('filename')
pg_wide_sample.index.name = 'Sample ID'
pg_wide_sample.columns.name = 'protein group'  

df = pg_wide.loc[pg_wide.sample_type=='sample'].drop(index_cols[1:], axis = 1).set_index('filename')
print(df.shape)

SELECT_FEAT=True
df = pg_wide_sample.copy()
freq_feat = pg_wide_sample.notna().sum()
freq_feat.head()  # training data
if SELECT_FEAT:
    # potentially this can take a few iterations to stabilize.
    df = filter_missingness(df, feat_prevalence=.6)
    df = filter_missingness(df=df, feat_prevalence=.6, axis=1)
df.shape

#### Impute missing values

In [ ]:
# overview of the distribution of missing values based on the non-missing intensity values of the protein in question
ax = pimmslearn.plotting.data.plot_feat_median_over_prop_missing(
    data=pg_wide.drop(index_cols, axis = 1), type='boxplot')

In [ ]:
# prepare data format
df_wide = pg_wide.reset_index().set_index(['filename'])[df.columns]
df_wide.index.name = 'Sample ID'  # already set
df_wide.columns.name = 'protein group'  # not set due to csv disk file format
print(df_wide.shape)

In [ ]:
val_X, train_X = pimmslearn.sampling.sample_data(df_wide.stack(),
                                           sample_index_to_drop=0,
                                           weights=df_wide.notna().sum(),
                                           frac=0.1,
                                           random_state=42,)
val_X, train_X = val_X.unstack(), train_X.unstack()
val_X = pd.DataFrame(pd.NA, index=train_X.index,
                     columns=train_X.columns).fillna(val_X)

model_selected = 'VAE'  # 'DAE'
model = AETransformer(
    model=model_selected,
    hidden_layers=[512,],
    latent_dim=50,
    out_folder='runs/scikit_interface',
    batch_size=10,
)

model.fit(train_X, val_X,
          epochs_max=50,
          cuda=False)

df_imputed = model.transform(train_X)

pred_val = val_X.stack().to_frame('observed')
pred_val[model_selected] = df_imputed.stack()
val_metrics = pimmslearn.models.calculte_metrics(pred_val, 'observed')

pd.DataFrame(val_metrics)

In [ ]:
df_imputed = df_imputed.replace(val_X)
df_imputed = df_imputed.stack()  # long-format
observed = df_imputed.loc[df.index]
imputed = df_imputed.loc[df_imputed.index.difference(df.index)]

In [ ]:
# plot distribution of missing value imputations
fig, axes = plt.subplots(2, figsize=(8, 4))

min_max = pimmslearn.plotting.data.get_min_max_iterable(
    [observed, imputed])
label_template = '{method} (N={n:,d})'
ax, _ = pimmslearn.plotting.data.plot_histogram_intensities(
    observed,
    ax=axes[0],
    min_max=min_max,
    label=label_template.format(method='measured',
                                n=len(observed),
                                ),
    color='grey',
    alpha=1)
_ = ax.legend()
ax, _ = pimmslearn.plotting.data.plot_histogram_intensities(
    imputed,
    ax=axes[1],
    min_max=min_max,
    label=label_template.format(method=f'{model_selected} imputed',
                                n=len(imputed),
                                ),
    color=color_model_mapping[model_selected],
    alpha=1)
_ = ax.legend()

In [ ]:
imputed_data = df_imputed.unstack()
processed_data = pd.merge(pg_wide[['filename','sample','ID','plate','sample_type']].set_index('filename'), imputed_data, left_index=True, right_index = True) 

#### Batch Correction

In [ ]:
index_cols = ['filename', 'Timepoints', 'sample', 'ID', 'Plate', 'Sample_type']
processed_data.set_index(index_cols, inplace = True)
corrected_data = pycombat_norm(processed_data.T,processed_data.reset_index()['Plate'].tolist()).T

### Rythmicity analysis

In [ ]:
df = corrected_data.copy()
df['Time']= df.Timepoints.copy()

df = df[['ID','Time']+protein_cols]
df.rename(columns = {'ID':'Individual_ID'}, inplace = True)

protein_cols = corrected_data.drop(['sample','plate','sample_type','ID','filename','Timepoints'], axis = 1).columns.tolist()



In [ ]:
# Apply z-score
df_zscored = df.groupby('Individual_ID').apply(zscore_by_individual, protein_cols)


In [ ]:
# Calculate Rythmicity

results_z = []

for protein_id in protein_cols:  # Loop over each protein column

    # Fit the cosinor model
    fit = fit_me(df_zscored['Time'], df_zscored[protein_id])

    # Extract values from the returned tuple
    regression_results, stats, rhythmic_params, part4, part5 = fit

    # Store relevant values
    results_z.append({
        'Protein_ID': protein_id,
        'p_value': stats['p'],  # Main p-value for rhythmicity
        'p_reject': stats['p_reject'],  # Alternative p-value
        'SNR': stats['SNR'],  # Signal-to-Noise Ratio
        'RSS': stats['RSS'],  # Residual Sum of Squares
        'resid_SE': stats['resid_SE'],  # Residual Standard Error
        'ME': stats['ME'],  # Maximum Error
        'Amplitude': rhythmic_params['amplitude'],  # Peak-to-trough difference
        'Acrophase': rhythmic_params['acrophase'],  # Time of peak
        'Mesor': rhythmic_params['mesor'],  # Baseline level
        'Period': rhythmic_params['period'],  # Expected period (should be ~24h)
        'Max_Location': rhythmic_params['max_loc'],  # Index of highest peak
        'Peaks': rhythmic_params['peaks'],  # Array of peak times
        'Peak_Heights': rhythmic_params['heights'],  # Heights of peaks
        'Troughs': rhythmic_params['troughs'],  # Array of lowest points
        'Trough_Heights': rhythmic_params['heights2'],  # Heights of troughs
        'Secondary_Period': rhythmic_params['period2']  # Alternative detected period
    })

In [ ]:
# Convert results_z to a DataFrame
results_z_df = pd.DataFrame(results_z)

# Multiple Testing Correction using Benjamini-Hochberg FDR
all_p_values = results_z_df['p_value'].values

# Apply Benjamini-Hochberg FDR correction
corrected_p_values = multipletests(all_p_values, method='fdr_bh')[1]

# Add corrected p-values to the DataFrame
results_z_df['Corrected_p_values'] = corrected_p_values

# Step 5: Filter for significant proteins (optional)
significant_proteins_z = results_z_df[results_z_df['Corrected_p_values'] < 0.05]

results_z_df[results_z_df['Corrected_p_values'] < 0.05].to_excel('supplementary_table.xlsx', index = None)


In [ ]:
print('The number of significantly rythmic proteins is: {}'.format(len(significant_proteins_z)))
# save the significant proteinnames:
significant_cosinorpy_z = results_z_df[results_z_df.Corrected_p_values<0.05]['Protein_ID'].tolist()


#### Plot heatmap

In [ ]:
results_z_df = pd.merge(proteome[['Protein.Group', 'Genes']], results_z_df, left_on='Protein.Group', right_on='Protein_ID')

results_z_df['Uniprot_ID']=results_z_df.Protein_ID.str.split(';').str[0]
results_z_df['Gene_ID']=results_z_df.Genes.str.split(';').str[0]

In [ ]:
heatmap_data = df_zscored[['Time','Individual_ID']+significant_cosinorpy_z].reset_index(drop=True).sort_values(['Time','Individual_ID']).set_index(['Time','Individual_ID']).T
heatmap_data_mean =df_zscored[['Time']+significant_cosinorpy_z].reset_index(drop=True).groupby('Time').mean().T
heatmap_data_mean = pd.merge(proteome[['Protein.Group','Genes']].drop_duplicates(), heatmap_data_mean, left_on = 'Protein.Group', right_index = True)
heatmap_data_mean.Genes = heatmap_data_mean.Genes.str.split(';').str[0]
heatmap_data_mean.set_index(['Protein.Group','Genes'], inplace = True)


In [ ]:
# plot
g = sns.clustermap(heatmap_data_mean,
                   figsize=(10, 50), #col_colors = group_colors, row_colors = cohort_cluster_colors,
                   col_cluster=False, row_cluster=True,
                   center = 0,
                   linewidth=0.5, linecolor='white', cmap = 'coolwarm')

g.figure.savefig("fig3_heatmap.svg")
g.figure.savefig("fig3_heatmap.pdf")


#### Plot Temporal Dynamics of Diurnal Plasma Protein Clusters

In [ ]:
# save the heatmap order and clusters
heatmap_order = heatmap_data_mean.iloc[g.dendrogram_row.reordered_ind].index.get_level_values("Protein.Group").tolist()
cluster1 = heatmap_order[0:47]
cluster2=heatmap_order[47:]


In [ ]:
# annotate cluster to the heatmap data
cluster_z = heatmap_data_mean.reset_index().melt(id_vars = ['Protein.Group','Genes'])
cluster_z['cluster']='cluster2'
cluster_z.loc[cluster_z['Protein.Group'].isin(cluster1),'cluster']='cluster1'
# plot the median protein abundance in each cluster with boxplots
fig = px.box(cluster_z, x = 'variable', y = 'value', facet_row = 'cluster', height = 500, width = 500,
       template='plotly_white')
fig.write_image('fig5a.svg')
fig.write_image('fig5a.pdf')
fig.show()

In [ ]:
# plot the line graphs displaying individual protein trajectories across 24 hoursin each cluster
fig = px.line(cluster_z, x='variable',y='value', color = 'Genes', facet_row = 'cluster',
              height = 500, width = 550,
              template='plotly_white')
fig.write_image('fig5b.svg')
fig.write_image('fig5b.pdf')
fig.show()

#### Extract statistics on each cluster

In [ ]:
# save the names of the proteins in each cluster
cluster2 = cluster_z[cluster_z.cluster == 'cluster2']['Protein.Group'].unique().tolist()
cluster1 = cluster_z[cluster_z.cluster == 'cluster1']['Protein.Group'].unique().tolist()

In [ ]:
# calculate mean peak time +sd for each cluster:
results_z_df['acrophase_hr']=(results_z_df.Acrophase/(2*np.pi))*24+9
new_peak = []
for _, row in results_z_df[['Acrophase', 'Amplitude']].iterrows():
    new_peak_is = acrophase_hr(row['Acrophase'], row['Amplitude'])+9
    if new_peak_is>24:
        new_peak_is = new_peak_is-24
    new_peak.append(new_peak_is) 
results_z_df['acrophase_hr_new'] = new_peak

print(len(cluster1))
print(results_z_df[results_z_df['Protein.Group'].isin(cluster1)].acrophase_hr_new.median())
print(results_z_df[results_z_df['Protein.Group'].isin(cluster1)].acrophase_hr_new.std())
print(len(cluster2))
print(results_z_df[results_z_df['Protein.Group'].isin(cluster2)].acrophase_hr_new.median())
print(results_z_df[results_z_df['Protein.Group'].isin(cluster2)].acrophase_hr_new.std())

In [ ]:
# calculate confidence intervals
sub = results_z_df[results_z_df['Protein.Group'].isin(cluster1+cluster2)].copy()
sub['cluster']='cluster1'
sub.loc[sub['Protein.Group'].isin(cluster2),'cluster']='cluster2'
result = sub.groupby('cluster')['acrophase_hr_new'].apply(mean_ci)

# Display result
print(result)

#### Plot the polar histogram illustrating the distribution of peak times (acrophases) for diurnal plasma proteins in the two clusters

In [ ]:
# Increase the number of bins for better resolution
num_bins = 48  # Adjust this for finer granularity
bins = np.linspace(0, 2*np.pi, num_bins + 1)  # Create bin edges

results_z_df['acrophase_radians_hr'] = results_z_df['acrophase_hr_new'] * (2 * np.pi / 24)
df_phase_only = results_z_df[results_z_df.Protein_ID.isin(significant_cosinorpy_z)].copy()
df_phase_only['cluster']='cluster1'
df_phase_only.loc[df_phase_only['Protein.Group'].isin(cluster2),'cluster'] = 'cluster2'

# Get unique clusters & generate color map
unique_clusters = df_phase_only['cluster'].unique()
color_map = {cluster: px.colors.qualitative.Plotly[i % len(px.colors.qualitative.Plotly)] 
             for i, cluster in enumerate(unique_clusters)}

# Step 2: Create bins for 48 bins (every 30 minutes, finer granularity)
num_bins = 48
bins = np.linspace(0, 2 * np.pi, num_bins + 1)  # Create bin edges for radians

# Create polar plot
fig = go.Figure()

# Loop through clusters & plot phase distributions
for cluster in unique_clusters:
    df_cluster = df_phase_only[df_phase_only['cluster'] == cluster]
    
    # Compute histogram for this cluster
    hist, bin_edges = np.histogram(df_cluster['acrophase_radians_hr'], bins=bins)
    
    # Compute bin centers in degrees
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2  
    bin_centers_deg = np.degrees(bin_centers)  

    # Add polar bar trace for this cluster
    fig.add_trace(go.Barpolar(
        r=hist,  
        theta=bin_centers_deg,  
        width=360 / num_bins,  
        name=f"Cluster {cluster}",
        marker_color=color_map[cluster],
        marker_line_color="black",
        marker_line_width=1.5,
        opacity=0.8
    ))

# Format layout
fig.update_layout(
    title="Phase Distribution of Rhythmic Proteins by Cluster",
    polar=dict(
        radialaxis=dict(showticklabels=False, ticks=""),  
        angularaxis=dict(direction="clockwise",
                         tickmode="array",
                         tickvals=np.linspace(0, 360, 12),  
                         ticktext=[f"{int(i)}h" for i in np.linspace(0, 24, 12)])
    ),
    template="plotly_white"
)

# Save and show plot
fig.write_image('fig5c.svg')
fig.write_image('fig5c.pdf')
fig.show()


In [ ]:
results_z_df[['Protein.Group','Acrophase','acrophase_hr_new']].acrophase_hr_new.describe()

In [ ]:
# In order to get meaningful amlitude values, calculate rythmicity for non-zscored data:
results = []  # List to store results

for protein_id in protein_cols:  # Loop over each protein column

    # Fit the cosinor model
    fit = fit_me(df['Time'], df[protein_id])

    # Extract values from the returned tuple
    regression_results, stats, rhythmic_params, part4, part5 = fit

    # Store relevant values
    results.append({
        'Protein_ID': protein_id,
        'p_value': stats['p'],  # Main p-value for rhythmicity
        'p_reject': stats['p_reject'],  # Alternative p-value
        'SNR': stats['SNR'],  # Signal-to-Noise Ratio
        'RSS': stats['RSS'],  # Residual Sum of Squares
        'resid_SE': stats['resid_SE'],  # Residual Standard Error
        'ME': stats['ME'],  # Maximum Error
        'Amplitude': rhythmic_params['amplitude'],  # Peak-to-trough difference
        'Acrophase': rhythmic_params['acrophase'],  # Time of peak
        'Mesor': rhythmic_params['mesor'],  # Baseline level
        'Period': rhythmic_params['period'],  # Expected period (should be ~24h)
        'Max_Location': rhythmic_params['max_loc'],  # Index of highest peak
        'Peaks': rhythmic_params['peaks'],  # Array of peak times
        'Peak_Heights': rhythmic_params['heights'],  # Heights of peaks
        'Troughs': rhythmic_params['troughs'],  # Array of lowest points
        'Trough_Heights': rhythmic_params['heights2'],  # Heights of troughs
        'Secondary_Period': rhythmic_params['period2']  # Alternative detected period
    })

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

### Plot the coefficient of variation and the relationship to amplitude and residual standard error

In [ ]:
# Coefficient of variation
df = processed_data.drop(['sample','plate','sample_type'], axis = 1)
individual_col='ID'
protein_cols = df.columns.difference([individual_col]).tolist()

cvs = []
for individual in df.ID.unique():
    cv_id = []
    for protein in protein_cols:
        cv_id.append(variation(df[df.ID==individual][protein]))
    cvs.append(cv_id)

cv = pd.DataFrame(cvs, columns = protein_cols).T
cv.columns = df.ID.unique().tolist()
cv_df = cv.mean(axis = 1).reset_index()
cv_df.columns = ['Protein.Group','CV']
cv_df = pd.merge(cv_df, proteome[['Protein.Group','Genes']].drop_duplicates(), on = 'Protein.Group')
cv_df['LFQ']=processed_data[protein_cols].mean().tolist()
cv_df['completeness']=list(df_wide.notnull().sum()/df_wide.shape[0])
cv_df.CV = cv_df.CV*100


In [ ]:
# results combined (to keep track of amplitude and phase)
result = pd.merge(results_df[['Protein_ID', 'Amplitude','resid_SE']], results_z_df[['Protein_ID','Gene_ID','Corrected_p_values','acrophase_hr_new']],on = 'Protein_ID')
cv_data = result.copy()
result = result[['Gene_ID','Protein_ID','Amplitude','acrophase_hr_new','Corrected_p_values']][result.Corrected_p_values<=0.05].copy()
result.columns = ['Gene_ID','Protein_ID','Amplitude (from original measurement)','Acrophase (from z-scored data)','Corrected p value']
result['Cluster']='cluster1'
result.loc[result.Protein_ID.isin(cluster2),'Cluster']='cluster2'
result.to_excel('supplementary_table.xlsx', index = None)


In [ ]:
# Plot CV's
cv_data['circadian'] = 'no'
cv_data.loc[cv_data.Protein_ID.isin(significant_cosinorpy_z),'circadian']='yes'
cv_data['-log10(pval)']=-1*np.log10(cv_data.Corrected_p_values)
cv_data = pd.merge(cv_data, cv_df, left_on = 'Protein_ID', right_on = 'Protein.Group')
cv_data['Name']=''
cv_data.loc[cv_data.circadian=='yes', 'Name'] = cv_data.Gene_ID

legend_data = cv_data.iloc[0:2,].copy()
legend_data.Amplitude = [0.1, 0.15]
legend_data.CV = 12
legend_data['-log10(pval)']=[(-np.log10(0.05)),(-np.log10(0.5))]
legend_data

fig = px.scatter(pd.concat([cv_data.sort_values('circadian', ascending = False), legend_data]), 
                 y = 'Amplitude', 
                 x = 'CV', 
                 #range_x=[0,1],
                 color = 'circadian', 
                 size = '-log10(pval)',
                 #text='Name',
                 template='none',
                 height = 400,
                 width=500)

fig.update_traces(textposition='middle right', 
                  textfont=dict(size=10, color='black'),
                  marker=dict(line=dict(width=0.5)))

fig.write_image('fig2b_amplitude_CV.svg') 
fig.write_image('fig2b_amplitude_CV.pdf') 
fig.show()


In [ ]:
# Pearson correlation test
corr, p_value = pearsonr(cv_data['CV'], cv_data['Amplitude'])

print(f"Pearson correlation coefficient: {corr:.4f}")
print(f"P-value: {p_value:.4g}")

In [ ]:
fig = px.scatter(pd.concat([cv_data.sort_values('circadian', ascending = False), legend_data]), 
                 y = 'resid_SE', 
                 x = 'CV', 
                 #range_x=[0,1],
                 color = 'circadian', 
                 size = '-log10(pval)',
                 #text='Name',
                 template='none',
                 height = 400,
                 width=500)

fig.update_traces(textposition='middle right', 
                  textfont=dict(size=10, color='black'),
                  marker=dict(line=dict(width=0.5)))

fig.write_image('fig2b_resid_CV.svg') 
fig.write_image('fig2b_resid_CV.pdf') 
fig.show()


In [ ]:
# Pearson correlation test
corr, p_value = pearsonr(cv_data['resid_SE'], cv_data['CV'])

print(f"Pearson correlation coefficient: {corr:.4f}")
print(f"P-value: {p_value:.4g}")

Calculate odds ratios

In [ ]:
# Step 1: Create a 'CV_group' column based on median
median_cv = cv_data['CV'].median()
cv_data['CV_group'] = cv_data['CV'].apply(lambda x: 'high' if x > median_cv else 'low')

# Step 2: Create a 2x2 contingency table
contingency = pd.crosstab(cv_data['circadian'], cv_data['CV_group'])
print("Contingency table:\n", contingency)

# Step 3: Perform Fisher's exact test
table = [
    [contingency.loc['yes', 'high'], contingency.loc['yes', 'low']],
    [contingency.loc['no', 'high'], contingency.loc['no', 'low']]
]

odds_ratio, p_value = fisher_exact(table)

print(f"\nFisher's Exact Test:")
print(f"Odds ratio: {odds_ratio:.4f}")
print(f"P-value: {p_value:.4g}")

In [ ]:
# Step 1: Create a 'Amplitude_group' column based on median
median_Amplitude = cv_data['Amplitude'].median()
cv_data['Amplitude_group'] = cv_data['Amplitude'].apply(lambda x: 'high' if x > median_Amplitude else 'low')

# Step 2: Create a 2x2 contingency table
contingency = pd.crosstab(cv_data['circadian'], cv_data['Amplitude_group'])
print("Contingency table:\n", contingency)

# Step 3: Perform Fisher's exact test
table = [
    [contingency.loc['yes', 'high'], contingency.loc['yes', 'low']],
    [contingency.loc['no', 'high'], contingency.loc['no', 'low']]
]

odds_ratio, p_value = fisher_exact(table)

print(f"\nFisher's Exact Test:")
print(f"Odds ratio: {odds_ratio:.4f}")
print(f"P-value: {p_value:.4g}")

# Enrichment analyses

Tissue and pathway enrichment analyses were conducted for identified protein clusters individually using the Database for Annotation, Visualization, and Integrated Discovery (DAVID) version, 2024 11,12. We provided a list of significant proteins under their official gene symbols (e.g., B2M, ALB) and used Homo sapiens (9606) as the background species. Tissue enrichment was performed with the UP_TISSUE database, excluding pathological, fetal, fluid-based (e.g., serum, plasma, cerebrospinal fluid), and placental enrichments. Pathway enrichment used the REACTOME_PATHWAY database, and we further supported these findings with Gene Ontology Biological Processes (GOTERM_BP_FAT) analysis. Enrichments with Benjamini–Hochberg–adjusted p-values below 0.05 were deemed significant.

In [ ]:
# Map data to uniprot and GO
uniprot_mapping = pd.read_csv('idmapping_2025_03_11.tsv', sep='\t')
# subset to reviewed variants
uniprot_mapping = uniprot_mapping[uniprot_mapping['Reviewed']=='reviewed'][['Entry','Entry Name', 'Protein names', 'Gene Names','Gene Ontology (biological process)']]
uniprot_mapping = uniprot_mapping[['Entry','Entry Name', 'Protein names', 'Gene Names','Gene Ontology (biological process)']]
uniprot_mapping['GO'] = uniprot_mapping['Gene Ontology (biological process)'].str.split('; ')
uniprot_mapping = uniprot_mapping.drop('Gene Ontology (biological process)', axis=1).explode('GO').drop_duplicates()
# add additional columns to the uniprot mapping for protein.Groups with multiple identifiers
additional_columns = []
for i in protein_cols:
    if ';' in i:
        multiple_ids = i.split(';')
        combined_df = uniprot_mapping[uniprot_mapping['Entry'].isin(multiple_ids)].astype(str)
        collapsed_df = combined_df.apply(lambda col: '; '.join(col)).to_frame().T
        collapsed_df.Entry = i
        additional_columns.append(collapsed_df)
additional_columns = pd.concat(additional_columns)
additional_columns['GO'] = additional_columns['GO'].str.split('; ')
additional_columns = additional_columns.explode('GO').drop_duplicates()
uniprot_mapping = pd.concat([uniprot_mapping, additional_columns]).reset_index(drop=True)
uniprot_mapping['Genes']=uniprot_mapping['Gene Names'].str.split(' ').str[0].str.split(';').str[0]
# subset to the list of significant proteins
mapping = pd.DataFrame(significant_cosinorpy_z, columns = ['Entry'])
mapping = pd.merge(mapping, uniprot_mapping[['Entry','Genes']].drop_duplicates(), on = 'Entry', how = 'left')

In [ ]:
# Tissue Enrichemnt Plot
tissue = pd.read_excel('Cluster1,2-Tissue.xlsx')
tissue['Genes'] = tissue['Genes'].str.split(', ')
tissue = tissue.explode('Genes')[['Term','Genes']].drop_duplicates()
bar_data = pd.DataFrame(tissue.Term.value_counts()).reset_index()
bar_data.rename(columns = {'index':'Term','count':'number of proteins'}, inplace = True)


In [ ]:
fig = px.bar(bar_data, y = 'Term', x = 'number of proteins', orientation = 'h',
       category_orders={'Term':bar_data.sort_values('number of proteins', ascending = False)['Term'].tolist()},
       width = 500, height = 500,
       template = 'plotly_white')
fig.write_image('fig4a.svg')
fig.write_image('fig4a.pdf')
fig.show()

In [ ]:
# plot tissue according to heatmap order
tissue_genes = tissue['Genes'].unique().tolist()

tissue_df = pd.merge(mapping, tissue, on = 'Genes', how = 'outer')
tissue_df.fillna('nan', inplace = True)

# # add missing Entries
tissue_df = pd.merge(pd.DataFrame(heatmap_data_mean.index.get_level_values("Protein.Group").tolist(), columns = ['Entry']), tissue_df[['Entry','Term','Genes']].drop_duplicates(), on = 'Entry', how = 'left')

heatmap_order = heatmap_data_mean.iloc[g.dendrogram_row.reordered_ind].index.get_level_values("Protein.Group").tolist()

# sort the tissue_df according to heatmap
tissue_df['Entry'] = pd.Categorical(tissue_df['Entry'], categories=heatmap_order, ordered=True)
tissue_df = tissue_df.sort_values(['Entry'])
tissue_df.Entry = tissue_df.Entry.astype(str)

# Get unique values of Genes and annotations
all_Genes = list(tissue_df['Genes'].unique())
all_annotations = list(tissue_df['Term'].unique())

# Create indices for sankey source (Geness) and targets (annotations)
tissue_df['Genes_index'] = tissue_df['Genes'].apply(lambda x: all_Genes.index(x))
tissue_df = tissue_df.sort_values(by='Genes_index').reset_index(drop=True)  # Ensure tissue_df is sorted by the order of 'all_Genes'
tissue_df['annotation_index'] = tissue_df['Term'].apply(lambda x: len(all_Genes) + all_annotations.index(x))


In [ ]:
df = tissue_df.copy()
df = df.replace('nan', np.nan)

# Count the number of terms per gene
df["TermCount"] = df.groupby("Genes")["Term"].transform("count")

# Normalize so that all bars have the same height
df["Proportion"] = 1 / df["TermCount"]

df["Term"] = df["Term"].fillna("No Term")  # Replace NaN with a label

# Plot using plotly express
fig = px.bar(df, 
             x="Proportion", 
             y="Genes", 
             color="Term", 
             orientation="h", 
             barmode="stack", 
             title="Stacked Bar Plot with Equal Heights per Gene",
             height = 2500, width = 300,
             category_orders={'Genes':tissue_df[['Genes','Genes_index']].drop_duplicates().sort_values('Genes_index').Genes.tolist()},
             template = 'plotly_white')

fig.write_image('fig3_tissue_stacked_bar.svg')
fig.write_image('fig3_tissue_stacked_bar.pdf')
fig.show()

# Reactome

In [ ]:
# read DAVID analysis result
pathway = pd.read_excel('Cluster1,2-Reactome.xlsx')
pathway['Genes'] = pathway['Genes'].str.split(', ')
pathway = pathway.explode('Genes')[['Cluster names','Genes']].drop_duplicates()

In [ ]:
# plot the bar chart of the number of proteins in each cluster
bar_data = pd.DataFrame(pathway['Cluster names'].value_counts()).reset_index()
bar_data.rename(columns = {'index':'Term','count':'number of proteins'}, inplace = True)
fig = px.bar(bar_data, y = 'Cluster names', x = 'number of proteins', orientation = 'h',
       category_orders={'Cluster names':bar_data.sort_values('number of proteins', ascending = False)['Cluster names'].tolist()},
       width = 1000, height = 700,
       template = 'plotly_white')  
fig.update_layout(showlegend=False)
fig.write_image('fig4b.svg')
fig.write_image('fig4b.pdf')
fig.show()

In [ ]:
# Plot sankey for heatmep
pathway_genes = pathway['Genes'].unique().tolist()

pathway_df = pd.merge(mapping, pathway, on = 'Genes', how = 'outer')
pathway_df.fillna('nan', inplace = True)

# # add missing Entries
pathway_df = pd.merge(pd.DataFrame(heatmap_data_mean.index.get_level_values("Protein.Group").tolist(), columns = ['Entry']), pathway_df[['Entry','Cluster names','Genes']].drop_duplicates(), on = 'Entry', how = 'left')
print(pathway_df.shape)

sub = pathway_df['Cluster names'].value_counts().reset_index()
pathway_df.loc[pathway_df['Cluster names'].isin(sub[sub['count']<=heatmap_data_mean.shape[0]*0.05]['Cluster names']),'Cluster names']='low'


heatmap_order = heatmap_data_mean.iloc[g.dendrogram_row.reordered_ind].index.get_level_values("Protein.Group").tolist()

# sort the pathway_df according to heatmap
pathway_df['Entry'] = pd.Categorical(pathway_df['Entry'], categories=heatmap_order, ordered=True)
pathway_df = pathway_df.sort_values(['Entry'])
pathway_df.Entry = pathway_df.Entry.astype(str)
#pathway_df['Genes'] = pathway_df['Genes'].str.split(' ', expand = True)[0] 


# Data for Sankey plot
import random
import plotly.graph_objects as go
import plotly.io as pio 

# Get unique values of Genes and annotations
all_Genes = list(pathway_df['Genes'].unique())
#all_Genes.reverse()
all_annotations = list(pathway_df['Cluster names'].unique())


# Create indices for sankey source (Geness) and targets (annotations)
pathway_df['Genes_index'] = pathway_df['Genes'].apply(lambda x: all_Genes.index(x))
pathway_df = pathway_df.sort_values(by='Genes_index').reset_index(drop=True)  # Ensure pathway_df is sorted by the order of 'all_Genes'
pathway_df['annotation_index'] = pathway_df['Cluster names'].apply(lambda x: len(all_Genes) + all_annotations.index(x))

# Generate a color for each annotation
colors = ['#'+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(len(all_annotations))]

# Map annotation index to colors
target_colors = {len(all_Genes) + i: colors[i] for i in range(len(all_annotations))}

# Create a list of colors for the links based on target node
link_colors = [target_colors[target] for target in pathway_df['annotation_index']]

# Define node positions for source nodes (x = 0.1 for all sources, y spaced evenly)
source_y_positions = [i / len(all_Genes) for i in range(len(all_Genes))]  # Evenly spaced source y positions
source_x_positions = [0.1] * len(all_Genes)  # Fixed x position for sources

# Only set positions for sources, leave target positions to be automatically placed
node_x_positions = source_x_positions + [None] * len(all_annotations)  # No x positions for targets
node_y_positions = source_y_positions + [None] * len(all_annotations)  # No y positions for targets

# Create Sankey diagram
fig = go.Figure(go.Sankey(
    node = dict(
        pad = 15,
        thickness = 20,
        line = dict(color = "black", width = 0.5),
        label = all_Genes + all_annotations,
        x = node_x_positions,
        y = node_y_positions
    ),
    link = dict(
        source = pathway_df['Genes_index'],  # starting points (Geness)
        target = pathway_df['annotation_index'],  # end points (annotations)
        value = [1] * len(pathway_df),  # each link has equal value (1 for each protein-GO link)
        color = link_colors  # set the color of links
    )
))

# Update layout and display the figure
fig.update_layout(title_text="Sankey Diagram of Protein Genes and Annotations", 
                  font_size=10,
                  height=800,
                  width=600)

# Define the path where you want to save the PDF
output_path = 'fig3_annotation_heatmap_pathway_annotation.svg'

# Save the figure as a PDF
pio.write_image(fig, output_path, format='svg')

fig.show()

#### GO enrichment comparison

In [ ]:
GOBP = pd.read_excel('GOBP-Clusters-Refined.xlsx')
GOBP['Genes'] = GOBP['Genes'].str.split(', ')
GOBP = GOBP.explode('Genes')[['Cluster names','Genes']].drop_duplicates()

In [ ]:

bar_data = pd.DataFrame(GOBP['Cluster names'].value_counts()).reset_index()
bar_data.rename(columns = {'index':'Term','count':'number of proteins'}, inplace = True)
px.bar(bar_data, y = 'Cluster names', x = 'number of proteins', orientation = 'h',category_orders={'Cluster names':bar_data.sort_values('number of proteins', ascending = False)['Cluster names'].tolist()}, template = 'plotly_white') 